In [27]:
import ipywidgets as widgets
from IPython.display import display
import pyttsx3

# Load API key
import os
import openai
os.environ['OPENAI_API_KEY'] = '*******************'
openai.api_key = os.environ.get('OPENAI_API_KEY')


# Step 1: Collect User Information
meal_types = widgets.SelectMultiple(
    options=['Breakfast', 'Lunch', 'Dinner', 'Snack'],
    value=['Breakfast', 'Lunch'],
    description='Meals:',
    disabled=False
)

diet_restrictions = widgets.Text(
    value='',
    placeholder='e.g., vegetarian, gluten-free',
    description='Diet:',
    disabled=False
)

specific_ingredients = widgets.Text(
    value='',
    placeholder='e.g., chicken, broccoli',
    description='Include:',
    disabled=False
)

exclude_ingredients = widgets.Text(
    value='',
    placeholder='e.g., peanuts, shellfish',
    description='Exclude:',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to submit your preferences',
    icon='check'
)

def on_submit_button_clicked(b):
    # Step 2: Generate Meal Plans
    
    ingredients = 'rice, chicken, vegetables, banana, spices, salt, pepper, palm oil'
    
    exact_ingredients = True  # This should be defined based on your logic
    
    ingredients = 'rice, chicken, vegetables, banana, spices, salt, pepper, palm oil'

    text = f'''
    Create a healthy and typical Nigerian daily meal plan for breakfast, lunch, and dinner based on the following ingredients: ```{ingredients}```.
    Your output should be in markdown format. 
    Follow the instructions below carefully.

    ### Instructions:
    1. {'Use ONLY the provided ingredients with salt, pepper, palm oil, banana, vegetables, rice, chicken, and spices.' if exact_ingredients else 'Feel free to incorporate the provided ingredients as a base and add other ingredients if you consider them necessary to enhance the flavor, nutritional value, or overall appeal of the recipes.'}  
    2. Specify the exact amount of each ingredient.
    3. Ensure that the total daily calorie intake is below 2000 kcal. 
    4. For each meal, explain each recipe, step by step, in clear and simple sentences. Use bullet points or numbers to organize the steps.
    5. For each meal, specify the total number of calories and the number of servings.
    6. For each meal, provide a concise and descriptive title that summarizes the main ingredients and flavors. The title should also be a valid DALL-E prompt to generate an original image for the meal.
    7. For each recipe, indicate the prep, cook, and total time.

    Before answering, make sure that you have followed the instructions listed above (points 1 to 7).
    The last line of your answer should be a string that contains ONLY the titles of the recipes and nothing more, with commas separating each title.
    Example of the last line of your answer: 
    'Spiced Rice and Vegetable Medley, Grilled Chicken with Plantain and Spinach, Pepper Soup with Yam and Greens'
    '''

    prompt = f"Create a meal plan with the following preferences:\n"
    prompt += f"{text}\n"
    prompt += f"Dietary Restrictions: {diet_restrictions.value}\n"
    prompt += f"Include Ingredients: {specific_ingredients.value}\n"
    prompt += f"Exclude Ingredients: {exclude_ingredients.value}\n"
    prompt += f"Meal Types: {', '.join(meal_types.value)}\n"
    prompt += "Recipe Titles:\n"

    
    from openai import OpenAI
    client = OpenAI()
    
    system_role = 'You are a professional Nigerian chef with many years of experience'
    response = client.chat.completions.create( 
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': prompt},
                    {'role': 'system', 'content': system_role}],
        temperature = 1,
        top_p = 1,
        frequency_penalty = 0,
        presence_penalty = 0)
    
    recipes = response.choices[0].message.content
    print("Generated Recipes:")
    print(recipes)
    
    # Step 3: Generate Images for Each Recipe
    
    recipe_titles = recipes.split('\n')[-1].split(', ')
    for title in recipe_titles:
        image_prompt = f"{title}"
        image_response = client.images.generate(
            model='dall-e-3',
            prompt=image_prompt,
            n=1,
            size="1024x1024",
            quality='standard',
            style='natural'
        )
        image_url = image_response.data[0].url
        display(widgets.HTML(f"<img src='{image_url}' width='400'>"))
    
    '''
    for meal in meal_types.value:
        image_prompt = f"A meal consisting of {meal}. {diet_restrictions.value}. Includes {specific_ingredients.value}. Excludes {exclude_ingredients.value}."
        image_response = client.images.generate(
            model = 'dall-e-3',
            prompt=image_prompt,
            n=1,
            size="1024x1024",
            quality = 'standard',
            style = 'natural'
        )
        image_url = image_response.data[0].url
        display(widgets.HTML(f"<img src='{image_url}' width='400'>"))
    '''
    
    
    # Step 4: Ask if the user wants the recipes read aloud
    tts_option = widgets.ToggleButtons(
        options=['Yes', 'No'],
        description='Read Aloud:',
        disabled=False,
        button_style=''
    )
    
    def on_tts_option_change(change):
        if change['new'] == 'Yes':
            
            prompt = f'''I will provide a recipe that will be spoken aloud.
            If necessary, you will adjust the recipe to make it more readable.
            You can also add an introduction and summary, but you won't change the ingredients, instructions,
            or other key elements of the recipe.
            Recipe:```{recipe}```'''
             
            from openai import OpenAI
            client = OpenAI()

            response = client.chat.completions.create(
                model='gpt-3.5-turbo',
                messages=[
                    {'role': 'user', 'content': prompt}
                ]
            )

            spoken_recipe = response.choices[0].message.content
            print(spoken_recipe)
    
            def speak(recipe):
                from openai import OpenAI
                client = OpenAI()
    
                response = client.audio.speech.create(
                    model='tts-1',
                    voice='alloy',
                    input=recipe)
                return response
    
            audio_response = speak(spoken_recipe)
        
        
            from IPython.display import Audio
            Audio(audio_response)
            
    tts_option.observe(on_tts_option_change, names='value')
    
    display(tts_option)

submit_button.on_click(on_submit_button_clicked)

# Display the widgets
display(meal_types, diet_restrictions, specific_ingredients, exclude_ingredients, submit_button)


SelectMultiple(description='Meals:', index=(0, 1), options=('Breakfast', 'Lunch', 'Dinner', 'Snack'), value=('…

Text(value='', description='Diet:', placeholder='e.g., vegetarian, gluten-free')

Text(value='', description='Include:', placeholder='e.g., chicken, broccoli')

Text(value='', description='Exclude:', placeholder='e.g., peanuts, shellfish')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit your preferences')

Generated Recipes:
### Breakfast: Banana Rice Porridge
- **Ingredients**:
  - 1 cup rice
  - 1 ripe banana
  - 2 cups water
  - 1/2 tsp salt
  - 1/4 tsp pepper
  - Cinnamon (optional)
- **Recipe**:
  1. Rinse the rice under cold water.
  2. In a pot, add the rice, water, salt, and pepper. 
  3. Bring to a boil, then reduce heat and let it simmer until rice is cooked.
  4. Mash the banana and stir it into the cooked rice.
  5. Sprinkle cinnamon if desired.
- **Total Calories**: 400 kcal
- **Number of Servings**: 2
- **Prep Time**: 5 minutes
- **Cook Time**: 20 minutes
- **Total Time**: 25 minutes

### Lunch: Spicy Chicken Stir-Fry with Vegetables
- **Ingredients**:
  - 300g chicken breast, sliced
  - 1 tbsp palm oil
  - 1/2 tsp salt
  - 1/4 tsp pepper
  - Mixed vegetables (carrots, bell peppers, green beans)
  - Spices (curry powder, thyme, paprika)
- **Recipe**:
  1. Heat palm oil in a pan and add sliced chicken. Cook until browned.
  2. Add salt, pepper, and spices to the chicken.
  3

HTML(value="<img src='https://oaidalleapiprodscus.blob.core.windows.net/private/org-ZVZ9NVIE5CJmOqXxV2AgEZN5/u…

HTML(value="<img src='https://oaidalleapiprodscus.blob.core.windows.net/private/org-ZVZ9NVIE5CJmOqXxV2AgEZN5/u…

HTML(value="<img src='https://oaidalleapiprodscus.blob.core.windows.net/private/org-ZVZ9NVIE5CJmOqXxV2AgEZN5/u…

ToggleButtons(description='Read Aloud:', options=('Yes', 'No'), value='Yes')